# Exercise: Supervised Learning

Recall our farming scenario, in which we want to look at how January temperatures have changed over time. Here, we will build a model that achieves this using supervised learning. 

With many libraries, this can be done in only a few lines of code. Here, we will break down the process into steps so that we can explore how things are working 'under the hood'.

## Four components
Recall that there are four key components to supervised learning: the data, the model, the cost function, and the optimiser. Let's inspect these one at a time.

### 1. The Data

We will be using publicly available weather data for Seattle. Let's load that and restrict it to January temperatures

In [1]:
!wget -P data https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/seattleWeather_1948-2017.csv

--2022-10-09 22:48:13--  https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/seattleWeather_1948-2017.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 762017 (744K) [text/plain]
Saving to: 'data/seattleWeather_1948-2017.csv'

     0K .......... .......... .......... .......... ..........  6%  450K 2s
    50K .......... .......... .......... .......... .......... 13% 1.78M 1s
   100K .......... .......... .......... .......... .......... 20%  841K 1s
   150K .......... .......... .......... .......... .......... 26%  956K 1s
   200K .......... .......... .......... .......... .......... 33%  968K 1s
   250K .......... .......... .......... .......... .......... 40%  652K 1s
   300K .......... .......... ......

In [2]:
import numpy as np
import pandas

import plotly.express as px
import plotly.graph_objects as go

# def warn(*args, **kwargs):
#     pass
# import warnings
# warnings.warn = warn

In [3]:
# Load a file containing weather data for seattle
data = pandas.read_csv('data/seattleWeather_1948-2017.csv', parse_dates=['date'])

# Keep only January temperatures
data = data[[d.month == 1 for d in data.date]].copy()


# Print the first and last few rows
# Remember that with Jupyter notebooks the last line of 
# code is automatically printed
data

,date,amount_of_precipitation,max_temperature,min_temperature,rain
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25229,2017-01-27,0.00,54,37,False
25230,2017-01-28,0.00,52,37,False
25231,2017-01-29,0.03,48,37,True
25232,2017-01-30,0.02,45,40,True


We have data from 1948 to 2017, split across 2170 rows. 

We will be analysing the relationship between `date` and daily minimum temperatures. Let's take a quick look at our data as a graph


In [4]:
# import graphing # custom graphing code. See our GitHub repository for details

# # Let's take a quick look at our data
# graphing.scatter_2D(data, label_x="date", label_y="min_temperature", title="January Temperatures (°F)")

fig = px.scatter(data, x="date", y="min_temperature",
                title="January Temperatures (°F)")
fig.show()

Machine learning usually works best when the X and Y axes have roughly the same range of values. We will cover why this is in later learning material. For now, lets just scale our data slightly.

In [5]:
# This block of code scales and offsets the data slightly, which helps the training process
# You do not need to understand this code. We will cover these concepts in later learning material

# Offset date into number of years since 1982
data["years_since_1982"] = [(d.year + d.timetuple().tm_yday / 365.25) - 1982 for d in data.date]

# Scale and offset temperature so that it has a smaller range of values
data["normalised_temperature"] = (data["min_temperature"] - np.mean(data["min_temperature"])) / np.std(data["min_temperature"])

# Graph
# graphing.scatter_2D(data, label_x="years_since_1982", label_y="normalised_temperature", title="January Temperatures (Normalised)")

fig = px.scatter(data, x="years_since_1982", y="normalised_temperature",
                title="January Temperatures (Normalised)")
fig.show()

### The model

We will be selecting a model called a simple linear regression model. This uses a line to make estimates. You might have come across trendlines like these before when making graphs.

In [6]:
class MyModel:

    def __init__(self):
        '''
        Creates a new MyModel
        '''
        # Straight lines described by two parameters:
        # The slop is the angle of the line
        self.slope = 0
        # The intercept moves the line up or down
        self.intercept = 0

    def predict(self, date):
        '''
        Estimates the temperature from the date
        '''
        return date * self.slope + self.intercept

# Create our model ready to be trained
model = MyModel()

print("Model made!")

Model made!


We wouldn't normally use a model before it has been trained, but for the sake of learning, let's take a quick look at it.

In [7]:
print(f"Model parameters before training: {model.intercept}, {model.slope}")

# Look at how well the model does before training
print("Model visualised before training:")

# graphing.scatter_2D(data, "years_since_1982", "normalised_temperature", trendline=model.predict)

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=data['years_since_1982'], y=data['normalised_temperature'],
                    mode='markers',
                    name='actual'))
fig.add_trace(go.Scatter(x=data['years_since_1982'], y=model.predict(data['years_since_1982']),
                    mode='lines+markers',
                    name='prediction'))
fig.update_xaxes(title_text='years_since_1982')
fig.update_yaxes(title_text='normalised_temperature')
fig.show()

Model parameters before training: 0, 0
Model visualised before training:


You can see that before training, our model (the red line) isn't useful at all. It always simply predicts zero.

### The objective (cost) function

Our next step is to create an _cost function_ (_objective function_).

These functions in supervised learning compare the model's estimate to the correct answer. In our case, our label is temperature, so our cost function will compare the estimated temperature to temperatures seen in the historical records.

In [8]:
def cost_function(actual_temperatures, estimated_temperatures):
    '''
    Calculates the difference between actual and estimated temperatures
    Returns the difference, and also returns the squared difference (the cost)

    actual_temperatures: One or more temperatures recorded in the past
    estimated_temperatures: Corresponding temperature(s) estimated by the model
    '''

    # Calculate the difference between actual temperatures and those
    # estimated by the model
    difference = estimated_temperatures - actual_temperatures

    # Convert to a single number that tells us how well the model did
    # (smaller numbers are better)
    cost = sum(difference ** 2)

    return difference, cost

### The optimiser

The role of the optimiser is to guess new parameter values for the model. 

We haven't covered optimisers in detail yet, so to make things simple, will use an pre-written optimiser. You do not need to understand how this works, but if you are curious, you can find it in our GitHub repository. 

In [9]:
'''
This describes is an optimiser used in module 0b
This hard codes some things to make the example code simple. In a normal
situation there would be many more options and nothing hard coded
'''
import numpy as np

class MyOptimizer:

    def calculate_gradient(self, model_input, diff):
        """
        This calculates gradient for a linear regession 
        using the SSD cost function:
        cost = sum((predicted - actual)^2)
        """

        # The partial derivatives of SSD are as follows
        # You don't need to be able to do this just yet but
        # it is important to note these give you the two gradients
        # that we need to train our model
        grad_slope = (diff * model_input).sum() * 2
        grad_intercept = diff.sum() * 2
        return grad_intercept, grad_slope


    def get_parameter_updates(self, model_inputs, cost, diff):

        # Calculate the gradient
        grad_intercept, grad_slope = self.calculate_gradient(model_inputs, diff)

        # Update the estimation of the line
        # We have hard coded some learning rates here
        # to keep things simple
        slope_update =     -grad_slope / 1000 / 2100 #5E6
        intercept_update = -grad_intercept / 1000 / 2100#5E6

        return intercept_update, slope_update 

In [10]:
# from m0b_optimizer import MyOptimizer

# Create an optimiser
optimizer = MyOptimizer()

## The Training Loop

Let's put these components together so that they train the model. 

Firt, let's make a function that performs one iteration of training. Read each step carefully below. If you would like, add some `print()` statements inside the method, to help you see the training in action.

In [11]:
def train_one_iteration(model_inputs, true_temperatures, last_cost:float):
    '''
    Runs a single iteration of training.


    model_inputs: One or more dates to provide the model (dates)
    true_temperatues: Corresponding temperatures known to occur on from those dates

    Returns:
        A boolean, as to whether training should continue
        The cost calculated (small numbers are better)
    '''

    # === USE THE MODEL ===
    # Estimate temperatures for all data that we have
    estimated_temperatures = model.predict(model_inputs)

    # === OBJECTIVE FUNCTION ===
    # Calculate how well the model is working
    # Smaller numbers are better 
    difference, cost = cost_function(true_temperatures, estimated_temperatures)

    # Decide whether to keep training
    # We will stop if the training is no longer improving the model effectively
    if cost >= last_cost:
        # Abort training
        return False, cost
    else:
        # === OPTIMISER ===
        # Calculate updates to parameters
        intercept_update, slope_update = optimizer.get_parameter_updates(model_inputs, cost, difference)

        # Change the model parameters
        model.slope += slope_update
        model.intercept += intercept_update

        return True, cost

print("Training method ready")

Training method ready


Let's run a few iterations manually, so that you can watch how training works.

Run the code below several times, and note how the model changes.

In [12]:
import math

print(f"Model parameters before training:\t\t{model.intercept:.8f},\t{model.slope:.8f}")

continue_loop, cost = train_one_iteration(model_inputs = data["years_since_1982"],
                                                    true_temperatures = data["normalised_temperature"],
                                                    last_cost = math.inf)

print(f"Model parameters after 1 iteration of training:\t{model.intercept:.8f},\t{model.slope:.8f}")


Model parameters before training:		0.00000000,	0.00000000
Model parameters after 1 iteration of training:	0.00000000,	0.01006832


It will take thousands of iterations to train the model well, so let's wrap it in a loop

In [13]:
# start the loop
print("Training beginning...")
last_cost = math.inf
i = 0
continue_loop = True
while continue_loop:

    # Run one iteration of training
    # This will tell us whether to stop training, and also what
    # the cost was for this iteration
    continue_loop, last_cost = train_one_iteration(model_inputs = data["years_since_1982"],
                                                    true_temperatures = data["normalised_temperature"],
                                                    last_cost = last_cost)
   
    # Print the status
    if i % 400 == 0:
        print("Iteration:", i)

    i += 1

    
print("Training complete!")
print(f"Model parameters after training:\t{model.intercept:.8f},\t{model.slope:.8f}")

Training beginning...
Iteration: 0
Iteration: 400
Iteration: 800
Iteration: 1200
Iteration: 1600
Iteration: 2000
Iteration: 2400
Iteration: 2800
Iteration: 3200
Iteration: 3600
Iteration: 4000
Training complete!
Model parameters after training:	-0.00648846,	0.01193327


In [14]:
# graphing.scatter_2D(data, "years_since_1982", "normalised_temperature", trendline=model.predict)

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=data['years_since_1982'], y=data['normalised_temperature'],
                    mode='markers',
                    name='actual'))
fig.add_trace(go.Scatter(x=data['years_since_1982'], y=model.predict(data['years_since_1982']),
                    mode='lines+markers',
                    name='prediction'))
fig.update_xaxes(title_text='years_since_1982')
fig.update_yaxes(title_text='normalised_temperature')
fig.show()

Notice how now the model is trained, it is giving more sensible predictions about January temperatures.

Interestingly, the model shows temperatures going up over time. Perhaps we need to stop feeding grain to our elk earlier in the year!

## Summary

In this exercise, we split up supervised learning into its individual stages to see what is going on 'under the hood' when we use 3rd party libraries. The important point to take away is how these pieces integrate together. In particular, how data is required by most parts of this process.